In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from functools import partial

import os
import xml.etree.ElementTree as ET

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
AUTO = tf.data.experimental.AUTOTUNE

In [2]:
# Path to the folders containing images and annotations
annotations_folder = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\annotations'
images_folder = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\images'

# Initialize lists to store image data and labels
images = []
labels = []

# Parse XML annotations and load images
for breed_folder in os.listdir(annotations_folder):
    breed_path = os.path.join(annotations_folder, breed_folder)
    for annotation_file in os.listdir(breed_path):
        annotation_path = os.path.join(breed_path, annotation_file)
        
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        breed = root.find('object/name').text
        
        img_path = os.path.join(images_folder, breed_folder, annotation_file + '.jpg')
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224, 224))  # Resize to desired size
        image = image/255
        
        images.append(image)
        labels.append(breed)

In [3]:
# Convert labels to numerical format
no_of_labels = (len(labels))
print(no_of_labels)

20580


In [4]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
labels_encoded=labels_encoded.reshape(no_of_labels, 1)
print(labels_encoded.shape)

(20580, 1)


In [5]:
# Preprocess the labels
labels_encoded_onehot = to_categorical(labels_encoded, num_classes)
print(labels_encoded_onehot.shape)


(20580, 120)


In [6]:
# Convert the images list to a NumPy array
images = np.array(images)
# print(images[0].shape)
print(images[0][1][223][0])


0.3137254901960784


In [7]:
# Print the first row
print("First row of labels_encoded_onehot:")
print(labels_encoded_onehot[0])

# Print the shape of the array
print("Shape of labels_encoded_onehot:", labels_encoded_onehot.shape)

First row of labels_encoded_onehot:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Shape of labels_encoded_onehot: (20580, 120)


In [8]:
print((labels_encoded_onehot.shape))
print((images.shape))

(20580, 120)
(20580, 224, 224, 3)


In [9]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded_onehot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print((y_train).shape)
print((X_train).shape)
print((y_val).shape)
print((X_val).shape)



MemoryError: Unable to allocate 3.46 GiB for an array with shape (3087, 224, 224, 3) and data type float64

In [ ]:
# Build Simple CNN
def build_simple_cnn(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional blocks
    for _ in range(5):
        model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))
    
    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
# Build Xception model with transfer learning
def build_xception(input_shape, num_classes):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Specify input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape

# Build and compile Simple CNN model
simple_cnn_model = build_simple_cnn(input_shape, num_classes)
simple_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Build and compile Xception model
xception_model = build_xception(input_shape, num_classes)
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Data augmentation for Simple CNN
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     horizontal_flip=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     fill_mode='nearest',
#     preprocessing_function=tf.keras.applications.xception.preprocess_input
# )

# # Create data generators using tf.data
# def create_data_generator(images, labels, batch_size, is_training=True):
#     dataset = tf.data.Dataset.from_tensor_slices((images, labels))
#     if is_training:
#         dataset = dataset.shuffle(buffer_size=len(images)).repeat()

#     dataset = dataset.map(lambda x, y: (datagen.random_transform(x), y), num_parallel_calls=tf.data.AUTOTUNE)
#     dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
#     return dataset

# # Create data generators
# train_data_generator = create_data_generator(X_train, y_train_onehot, batch_size=32)
# val_data_generator = create_data_generator(X_val, y_val_onehot, batch_size=32, is_training=False)
# test_data_generator = create_data_generator(X_test, y_test_onehot, batch_size=32, is_training=False)

# Train Simple CNN
history_simple_cnn = simple_cnn_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

# Train Xception
history_xception = xception_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

# Evaluate models
simple_cnn_loss, simple_cnn_acc = simple_cnn_model.evaluate(X_test, y_test, verbose=2)
xception_loss, xception_acc = xception_model.evaluate(X_test, y_test, verbose=2)

print("Simple CNN - Test accuracy:", simple_cnn_acc)
print("Xception - Test accuracy:", xception_acc)

In [ ]:
simple_cnn_model.summary()

In [ ]:
xception_model.summary()

In [ ]:
# Plot accuracy and loss curves for Simple CNN and Xception
def plot_training_curves(history, title):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.title(title + ' Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title(title + ' Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_curves(history_simple_cnn, 'Simple CNN')
plot_training_curves(history_xception, 'Xception')

# Load and preprocess a new image
new_image_path = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\images - Copy\n02085936_37.jpg'
new_image = cv2.imread(new_image_path)
new_image = cv2.resize(new_image, (224, 224))
new_image = new_image / 255.0
new_image = np.expand_dims(new_image, axis=0)

# Make predictions using the trained models
simple_cnn_predictions = simple_cnn_model.predict(new_image)
xception_predictions = xception_model.predict(new_image)

# Decode the predictions
simple_cnn_predicted_class = np.argmax(simple_cnn_predictions)
xception_predicted_class = np.argmax(xception_predictions)

simple_cnn_predicted_label = label_encoder.classes_[simple_cnn_predicted_class]
xception_predicted_label = label_encoder.classes_[xception_predicted_class]

simple_cnn_probabilities = simple_cnn_predictions[0]
xception_probabilities = xception_predictions[0]

# Print the predictions
print("Simple CNN Predicted Class:", simple_cnn_predicted_label)
print("Simple CNN Predicted Probabilities:", simple_cnn_probabilities)

print("Xception Predicted Class:", xception_predicted_label)
print("Xception Predicted Probabilities:", xception_probabilities)

# Plot the predicted probabilities for the new image
def plot_predicted_probabilities(classes, probabilities, title):
    plt.figure(figsize=(8, 4))
    plt.bar(classes, probabilities)
    plt.title(title)
    plt.xlabel('Classes')
    plt.ylabel('Probabilities')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

class_names = label_encoder.classes_
plot_predicted_probabilities(class_names, simple_cnn_probabilities, 'Simple CNN Predicted Probabilities')
plot_predicted_probabilities(class_names, xception_probabilities, 'Xception Predicted Probabilities')